In [1]:
%run Runge_Adams_FDA.ipynb

In [2]:
eps, t, h, a, b = symbols(r'\varepsilon, t, h, alpha beta', real=True)
u, u1 = symbols('u, u1', cls=Function)

In [3]:
eq = eps*u(t).diff(t, 2) + (1 + eps**2)*u(t).diff(t) + (1 - eps**2)*u(t)
print(latex(eq))

\varepsilon \frac{d^{2}}{d t^{2}} u{\left(t \right)} + \left(1 - \varepsilon^{2}\right) u{\left(t \right)} + \left(\varepsilon^{2} + 1\right) \frac{d}{d t} u{\left(t \right)}


## Nayfeh A. H. Introduction to Perturbation Techniques, Wiley-VCH, 1st edition, 1993. 536 p.

In [4]:
s = dsolve(eq, u(t), ics={u(t).subs(t, 0): 0, u(t).subs(t, 1): 1})
print(s.args[1].expand().simplify())

(-exp(t/\varepsilon) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(-exp(1/\varepsilon) + exp(\varepsilon + 2))


In [5]:
sol = {u(t): s.args[1], u1(t): s.args[1].diff(t)}

In [6]:
print(s.args[1].expand().factor())

E*(exp(2*t)*exp(\varepsilon*t) - exp(t/\varepsilon))*exp(1/\varepsilon)*exp(\varepsilon)*exp(-t)*exp(-t/\varepsilon)*exp(-\varepsilon*t)/(-exp(1/\varepsilon) + exp(2)*exp(\varepsilon))


In [7]:
s.args[1].diff(t).subs(t, 0).expand().factor()

E*(\varepsilon**2 + 2*\varepsilon - 1)*exp(1/\varepsilon)*exp(\varepsilon)/(\varepsilon*(-exp(1/\varepsilon) + exp(2)*exp(\varepsilon)))

In [8]:
print(latex(s.args[1].diff(t).subs(t, 0).expand().factor()))

\frac{e \left(\varepsilon^{2} + 2 \varepsilon - 1\right) e^{\frac{1}{\varepsilon}} e^{\varepsilon}}{\varepsilon \left(- e^{\frac{1}{\varepsilon}} + e^{2} e^{\varepsilon}\right)}


In [9]:
print(ccode(s.args[1].diff(t).subs(t, 0).expand().factor()))

M_E*(pow(\varepsilon, 2) + 2*\varepsilon - 1)*exp(1.0/\varepsilon)*exp(\varepsilon)/(\varepsilon*(-exp(1.0/\varepsilon) + exp(2)*exp(\varepsilon)))


In [10]:
init((u(t), u1(t)), t, h)

In [11]:
set_clip(8, 7, Rational(0, 1))

In [12]:
expand((T(u(t+h))-T(u(t-h)))/(2*h))

h**6*Derivative(u(t), (t, 7))/5040 + h**4*Derivative(u(t), (t, 5))/120 + h**2*Derivative(u(t), (t, 3))/6 + Derivative(u(t), t)

In [13]:
def g(y):
    return Matrix([\
        y[1],\
        -((1 + eps**2)*y[1] + (1 - eps**2)*y[0])/eps,\
    ])
    
def f(y):
    return Matrix([\
        clip_n(y[1]),\
        clip_n(-((1 + eps**2)*y[1] + (1 - eps**2)*y[0])/eps),\
    ])

s = (('u(t)', 'y[0]'), ('u1(t)', 'y[1]'), (r'\varepsilon', 'eps'))
def replace(r):
    for a, b in s:
        r = r.replace(a, b)
    return r

In [14]:
start = time.time()

## Gauss–Legendre method

In [15]:
set_clip(5, 4, Rational(0, 1))
r = GaussLegendre2(f, Matrix([u(t), u1(t)]))

In [16]:
F1 = clip((T(u(t+h))-T(u(t)))/h - T(r[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


u_{tt}/2

h^2 =>


u_{ttt}/6

h^3 =>


u_{tttt}/24

- u_{1} + u_{t}
+h\left(
\frac{u_{tt}}{2}
\right)
+h^2\left(
\frac{u_{ttt}}{6}
\right)
+h^3\left(
\frac{u_{tttt}}{24}
\right)


In [17]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h - T(r[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


u1_{tt}/2

h^2 =>


u1_{ttt}/6

h^3 =>


u1_{tttt}/24

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{u1_{tt}}{2}
\right)
+h^2\left(
\frac{u1_{ttt}}{6}
\right)
+h^3\left(
\frac{u1_{tttt}}{24}
\right)


In [18]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


\varepsilon*(u - u1)/2 - (u + u1)/(2*\varepsilon)

h^2 =>


\varepsilon**2*(u - u1)/3 - \varepsilon*u1/3 - 2*u1/3 + u1/(3*\varepsilon) - (u + u1)/(3*\varepsilon**2)

h^3 =>


\varepsilon**3*(u - u1)/4 + \varepsilon**2*(u - 2*u1)/4 + \varepsilon*(u - 3*u1)/4 - u/2 - (u + 3*u1)/(4*\varepsilon) + (u + 2*u1)/(4*\varepsilon**2) - (u + u1)/(4*\varepsilon**3)

- u_{1} + u_{t}
+h\left(
\frac{\varepsilon \left(u - u_{1}\right)}{2} - \frac{u + u_{1}}{2 \varepsilon}
\right)
+h^2\left(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{3} - \frac{\varepsilon u_{1}}{3} - \frac{2 u_{1}}{3} + \frac{u_{1}}{3 \varepsilon} - \frac{u + u_{1}}{3 \varepsilon^{2}}
\right)
+h^3\left(
\frac{\varepsilon^{3} \left(u - u_{1}\right)}{4} + \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{4} + \frac{\varepsilon \left(u - 3 u_{1}\right)}{4} - \frac{u}{2} - \frac{u + 3 u_{1}}{4 \varepsilon} + \frac{u + 2 u_{1}}{4 \varepsilon^{2}} - \frac{u + u_{1}}{4 \varepsilon^{3}}
\right)


In [19]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


-\varepsilon**2*(u - u1)/2 + \varepsilon*u1/2 + u1 - u1/(2*\varepsilon) + (u + u1)/(2*\varepsilon**2)

h^2 =>


-\varepsilon**3*(u - u1)/3 - \varepsilon**2*(u - 2*u1)/3 - \varepsilon*(u - 3*u1)/3 + 2*u/3 + (u + 3*u1)/(3*\varepsilon) - (u + 2*u1)/(3*\varepsilon**2) + (u + u1)/(3*\varepsilon**3)

h^3 =>


-\varepsilon**4*(u - u1)/4 - \varepsilon**3*(2*u - 3*u1)/4 - \varepsilon**2*(2*u - 5*u1)/4 + \varepsilon*(2*u + 3*u1)/4 + u1 + (2*u - 3*u1)/(4*\varepsilon) + (2*u + 5*u1)/(4*\varepsilon**2) - (2*u + 3*u1)/(4*\varepsilon**3) + (u + u1)/(4*\varepsilon**4)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
- \frac{\varepsilon^{2} \left(u - u_{1}\right)}{2} + \frac{\varepsilon u_{1}}{2} + u_{1} - \frac{u_{1}}{2 \varepsilon} + \frac{u + u_{1}}{2 \varepsilon^{2}}
\right)
+h^2\left(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{3} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{3} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{3} + \frac{2 u}{3} + \frac{u + 3 u_{1}}{3 \varepsilon} - \frac{u + 2 u_{1}}{3 \varepsilon^{2}} + \frac{u + u_{1}}{3 \varepsilon^{3}}
\right)
+h^3\left(
- \frac{\varepsilon^{4} \left(u - u_{1}\right)}{4} - \frac{\varepsilon^{3} \left(2 u - 3 u_{1}\right)}{4} - \frac{\varepsilon^{2} \left(2 u - 5 u_{1}\right)}{4} + \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{4} + u_{1} + \frac{2 u - 3 u_{1}}{4 \varepsilon} + \frac{2 u + 5 u_{1}}{4 \varepsilon^{2}} - \frac{2 u + 3 u_{1}}{4 \varepsilon^{3}} + \frac{u + u_{1}}{4 \varepsilon^{4}}
\right)


In [20]:
f1[2].subs(sol).expand().simplify()

(-\varepsilon**6*exp(t/\varepsilon) - 3*\varepsilon**5*exp(t/\varepsilon) - 3*\varepsilon**4*exp(t/\varepsilon) - \varepsilon**3*exp(t/\varepsilon) - \varepsilon**3*exp(t*(\varepsilon + 2)) + 3*\varepsilon**2*exp(t*(\varepsilon + 2)) - 3*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(3*\varepsilon**3*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [21]:
r = f1[2].subs(sol).expand().simplify()
r.subs(t, solve(r.diff(t), t)[0]).expand().simplify()

(\varepsilon**3*(-exp((2*\varepsilon**2 + \varepsilon*(\varepsilon**2 + 2*\varepsilon - 1) + \varepsilon*log((\varepsilon**4 - 4*\varepsilon**3 + 6*\varepsilon**2 - 4*\varepsilon + 1)/(\varepsilon**4*(\varepsilon**4 + 4*\varepsilon**3 + 6*\varepsilon**2 + 4*\varepsilon + 1))) + 4*\varepsilon - 2)/(\varepsilon**2 + 2*\varepsilon - 1)) + exp((\varepsilon**2*log((\varepsilon**4 - 4*\varepsilon**3 + 6*\varepsilon**2 - 4*\varepsilon + 1)/(\varepsilon**4*(\varepsilon**4 + 4*\varepsilon**3 + 6*\varepsilon**2 + 4*\varepsilon + 1))) + \varepsilon**2 + 2*\varepsilon - 1)/(\varepsilon*(\varepsilon**2 + 2*\varepsilon - 1))))*(\varepsilon**3 + 3*\varepsilon**2 + 3*\varepsilon + 1)*exp((\varepsilon**3*log((\varepsilon**4 - 4*\varepsilon**3 + 6*\varepsilon**2 - 4*\varepsilon + 1)/(\varepsilon**4*(\varepsilon**4 + 4*\varepsilon**3 + 6*\varepsilon**2 + 4*\varepsilon + 1))) + \varepsilon**2*(\varepsilon**2 + 2*\varepsilon - 1) + \varepsilon**2*log((\varepsilon**4 - 4*\varepsilon**3 + 6*\varepsilon**2 - 

In [22]:
f2[2].subs(sol).expand().simplify()

(\varepsilon**8*exp(t/\varepsilon) + 4*\varepsilon**7*exp(t/\varepsilon) + 6*\varepsilon**6*exp(t/\varepsilon) + 4*\varepsilon**5*exp(t/\varepsilon) + \varepsilon**4*exp(t/\varepsilon) - \varepsilon**4*exp(t*(\varepsilon + 2)) + 4*\varepsilon**3*exp(t*(\varepsilon + 2)) - 6*\varepsilon**2*exp(t*(\varepsilon + 2)) + 4*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(3*\varepsilon**4*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [23]:
print(latex(f2[2].subs(sol).expand().simplify()))

\frac{\left(\varepsilon^{8} e^{\frac{t}{\varepsilon}} + 4 \varepsilon^{7} e^{\frac{t}{\varepsilon}} + 6 \varepsilon^{6} e^{\frac{t}{\varepsilon}} + 4 \varepsilon^{5} e^{\frac{t}{\varepsilon}} + \varepsilon^{4} e^{\frac{t}{\varepsilon}} - \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} + 4 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} - 6 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} + 4 \varepsilon e^{t \left(\varepsilon + 2\right)} - e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{3 \varepsilon^{4} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


## Crank–Nicolson method

In [24]:
set_clip(5, 4, Rational(0, 1))
r = CrankNicolson2(f, Matrix([u(t), u1(t)]))

In [25]:
F1 = clip((T(u(t+h))-T(u(t)))/h - T(r[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


u_{tt}/2

h^2 =>


u_{ttt}/6

h^3 =>


u_{tttt}/24

- u_{1} + u_{t}
+h\left(
\frac{u_{tt}}{2}
\right)
+h^2\left(
\frac{u_{ttt}}{6}
\right)
+h^3\left(
\frac{u_{tttt}}{24}
\right)


In [26]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h - T(r[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


u1_{tt}/2

h^2 =>


u1_{ttt}/6

h^3 =>


u1_{tttt}/24

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{u1_{tt}}{2}
\right)
+h^2\left(
\frac{u1_{ttt}}{6}
\right)
+h^3\left(
\frac{u1_{tttt}}{24}
\right)


In [27]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


\varepsilon*(u - u1)/2 - (u + u1)/(2*\varepsilon)

h^2 =>


\varepsilon**2*(u - u1)/3 - \varepsilon*u1/3 - 2*u1/3 + u1/(3*\varepsilon) - (u + u1)/(3*\varepsilon**2)

h^3 =>


\varepsilon**3*(u - u1)/4 + \varepsilon**2*(u - 2*u1)/4 + \varepsilon*(u - 3*u1)/4 - u/2 - (u + 3*u1)/(4*\varepsilon) + (u + 2*u1)/(4*\varepsilon**2) - (u + u1)/(4*\varepsilon**3)

- u_{1} + u_{t}
+h\left(
\frac{\varepsilon \left(u - u_{1}\right)}{2} - \frac{u + u_{1}}{2 \varepsilon}
\right)
+h^2\left(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{3} - \frac{\varepsilon u_{1}}{3} - \frac{2 u_{1}}{3} + \frac{u_{1}}{3 \varepsilon} - \frac{u + u_{1}}{3 \varepsilon^{2}}
\right)
+h^3\left(
\frac{\varepsilon^{3} \left(u - u_{1}\right)}{4} + \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{4} + \frac{\varepsilon \left(u - 3 u_{1}\right)}{4} - \frac{u}{2} - \frac{u + 3 u_{1}}{4 \varepsilon} + \frac{u + 2 u_{1}}{4 \varepsilon^{2}} - \frac{u + u_{1}}{4 \varepsilon^{3}}
\right)


In [28]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


-\varepsilon**2*(u - u1)/2 + \varepsilon*u1/2 + u1 - u1/(2*\varepsilon) + (u + u1)/(2*\varepsilon**2)

h^2 =>


-\varepsilon**3*(u - u1)/3 - \varepsilon**2*(u - 2*u1)/3 - \varepsilon*(u - 3*u1)/3 + 2*u/3 + (u + 3*u1)/(3*\varepsilon) - (u + 2*u1)/(3*\varepsilon**2) + (u + u1)/(3*\varepsilon**3)

h^3 =>


-\varepsilon**4*(u - u1)/4 - \varepsilon**3*(2*u - 3*u1)/4 - \varepsilon**2*(2*u - 5*u1)/4 + \varepsilon*(2*u + 3*u1)/4 + u1 + (2*u - 3*u1)/(4*\varepsilon) + (2*u + 5*u1)/(4*\varepsilon**2) - (2*u + 3*u1)/(4*\varepsilon**3) + (u + u1)/(4*\varepsilon**4)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
- \frac{\varepsilon^{2} \left(u - u_{1}\right)}{2} + \frac{\varepsilon u_{1}}{2} + u_{1} - \frac{u_{1}}{2 \varepsilon} + \frac{u + u_{1}}{2 \varepsilon^{2}}
\right)
+h^2\left(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{3} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{3} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{3} + \frac{2 u}{3} + \frac{u + 3 u_{1}}{3 \varepsilon} - \frac{u + 2 u_{1}}{3 \varepsilon^{2}} + \frac{u + u_{1}}{3 \varepsilon^{3}}
\right)
+h^3\left(
- \frac{\varepsilon^{4} \left(u - u_{1}\right)}{4} - \frac{\varepsilon^{3} \left(2 u - 3 u_{1}\right)}{4} - \frac{\varepsilon^{2} \left(2 u - 5 u_{1}\right)}{4} + \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{4} + u_{1} + \frac{2 u - 3 u_{1}}{4 \varepsilon} + \frac{2 u + 5 u_{1}}{4 \varepsilon^{2}} - \frac{2 u + 3 u_{1}}{4 \varepsilon^{3}} + \frac{u + u_{1}}{4 \varepsilon^{4}}
\right)


In [29]:
f1[2].subs(sol).expand().simplify()

(-\varepsilon**6*exp(t/\varepsilon) - 3*\varepsilon**5*exp(t/\varepsilon) - 3*\varepsilon**4*exp(t/\varepsilon) - \varepsilon**3*exp(t/\varepsilon) - \varepsilon**3*exp(t*(\varepsilon + 2)) + 3*\varepsilon**2*exp(t*(\varepsilon + 2)) - 3*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(3*\varepsilon**3*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [30]:
f2[2].subs(sol).expand().simplify()

(\varepsilon**8*exp(t/\varepsilon) + 4*\varepsilon**7*exp(t/\varepsilon) + 6*\varepsilon**6*exp(t/\varepsilon) + 4*\varepsilon**5*exp(t/\varepsilon) + \varepsilon**4*exp(t/\varepsilon) - \varepsilon**4*exp(t*(\varepsilon + 2)) + 4*\varepsilon**3*exp(t*(\varepsilon + 2)) - 6*\varepsilon**2*exp(t*(\varepsilon + 2)) + 4*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(3*\varepsilon**4*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [31]:
print(latex(f2[2].subs(sol).expand().simplify()))

\frac{\left(\varepsilon^{8} e^{\frac{t}{\varepsilon}} + 4 \varepsilon^{7} e^{\frac{t}{\varepsilon}} + 6 \varepsilon^{6} e^{\frac{t}{\varepsilon}} + 4 \varepsilon^{5} e^{\frac{t}{\varepsilon}} + \varepsilon^{4} e^{\frac{t}{\varepsilon}} - \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} + 4 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} - 6 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} + 4 \varepsilon e^{t \left(\varepsilon + 2\right)} - e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{3 \varepsilon^{4} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


## "original" Runge–Kutta method

In [32]:
set_clip(7, 6, Rational(0, 1))
r = RungeKutta4(f, Matrix([u(t), u1(t)]))

In [33]:
F1 = clip((T(u(t+h))-T(u(t)))/h - T(r[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-\varepsilon*(u - u1)/2 + u_{tt}/2 + (u + u1)/(2*\varepsilon)

h^2 =>


...

h^4 =>


u_{ttttt}/120

h^5 =>


u_{tttttt}/720

- u_{1} + u_{t}
+h\left(
- \frac{\varepsilon \left(u - u_{1}\right)}{2} + \frac{u_{tt}}{2} + \frac{u + u_{1}}{2 \varepsilon}
\right)
+h^2\left(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{6} - \frac{\varepsilon u_{1}}{6} - \frac{2 u_{1} - u_{ttt}}{6} + \frac{u_{1}}{6 \varepsilon} - \frac{u + u_{1}}{6 \varepsilon^{2}}
\right)
+h^3\left(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{24} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{24} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{24} + \frac{2 u + u_{tttt}}{24} + \frac{u + 3 u_{1}}{24 \varepsilon} - \frac{u + 2 u_{1}}{24 \varepsilon^{2}} + \frac{u + u_{1}}{24 \varepsilon^{3}}
\right)
+h^4\left(
\frac{u_{ttttt}}{120}
\right)
+h^5\left(
\frac{u_{tttttt}}{720}
\right)


In [34]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h - T(r[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


\varepsilon**2*(u - u1)/2 - \varepsilon*u1/2 - (2*u1 - u1_{tt})/2 + u1/(2*\varepsilon) - (u + u1)/(2*\varepsilon**2)

h^2 =>


...

h^4 =>


u1_{ttttt}/120

h^5 =>


u1_{tttttt}/720

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{2} - \frac{\varepsilon u_{1}}{2} - \frac{2 u_{1} - u1_{tt}}{2} + \frac{u_{1}}{2 \varepsilon} - \frac{u + u_{1}}{2 \varepsilon^{2}}
\right)
+h^2\left(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{6} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{6} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{6} + \frac{2 u + u1_{ttt}}{6} + \frac{u + 3 u_{1}}{6 \varepsilon} - \frac{u + 2 u_{1}}{6 \varepsilon^{2}} + \frac{u + u_{1}}{6 \varepsilon^{3}}
\right)
+h^3\left(
\frac{\varepsilon^{4} \left(u - u_{1}\right)}{24} + \frac{\varepsilon^{3} \left(2 u - 3 u_{1}\right)}{24} + \frac{\varepsilon^{2} \left(2 u - 5 u_{1}\right)}{24} - \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{24} - \frac{4 u_{1} - u1_{tttt}}{24} - \frac{2 u - 3 u_{1}}{24 \varepsilon} - \frac{2 u + 5 u_{1}}{24 \varepsilon^{2}} + \frac{2 u + 3 u_{1}}{24 \varepsilon^{3}} - \frac{u + u_{1}}{24 \varepsilo

In [35]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-\varepsilon**4*(u - u1)/120 - \varepsilon**3*(2*u - 3*u1)/120 - \varepsilon**2*(2*u - 5*u1)/120 + \varepsilon*(2*u + 3*u1)/120 + u1/30 + (2*u - 3*u1)/(120*\varepsilon) + (2*u + 5*u1)/(120*\varepsilon**2) - (2*u + 3*u1)/(120*\varepsilon**3) + (u + u1)/(120*\varepsilon**4)

h^5 =>


-\varepsilon**5*(u - u1)/144 - \varepsilon**4*(3*u - 4*u1)/144 - \varepsilon**3*(u - 2*u1)/36 + \varepsilon**2*u1/18 + \varepsilon*(u + 7*u1)/144 + u/24 - (u - 7*u1)/(144*\varepsilon) - u1/(18*\varepsilon**2) + (u + 2*u1)/(36*\varepsilon**3) - (3*u + 4*u1)/(144*\varepsilon**4) + (u + u1)/(144*\varepsilon**5)

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{\varepsilon^{4} \left(u - u_{1}\right)}{120} - \frac{\varepsilon^{3} \left(2 u - 3 u_{1}\right)}{120} - \frac{\varepsilon^{2} \left(2 u - 5 u_{1}\right)}{120} + \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{120} + \frac{u_{1}}{30} + \frac{2 u - 3 u_{1}}{120 \varepsilon} + \frac{2 u + 5 u_{1}}{120 \varepsilon^{2}} - \frac{2 u + 3 u_{1}}{120 \varepsilon^{3}} + \frac{u + u_{1}}{120 \varepsilon^{4}}
\right)
+h^5\left(
- \frac{\varepsilon^{5} \left(u - u_{1}\right)}{144} - \frac{\varepsilon^{4} \left(3 u - 4 u_{1}\right)}{144} - \frac{\varepsilon^{3} \left(u - 2 u_{1}\right)}{36} + \frac{\varepsilon^{2} u_{1}}{18} + \frac{\varepsilon \left(u + 7 u_{1}\right)}{144} + \frac{u}{24} - \frac{u - 7 u_{1}}{144 \varepsilon} - \frac{u_{1}}{18 \varepsilon^{2}} + \frac{u + 2 u_{1}}{36 \varepsilon^{3}} - \frac{3 u + 4 u_{1}}{144 \varepsilon^{4}} + \frac{u + u_{1}}{144 \varepsilon^{5}}
\right)


In [36]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^4 =>


\varepsilon**5*(u - u1)/120 + \varepsilon**4*(3*u - 4*u1)/120 + \varepsilon**3*(u - 2*u1)/30 - \varepsilon**2*u1/15 - \varepsilon*(u + 7*u1)/120 - u/20 + (u - 7*u1)/(120*\varepsilon) + u1/(15*\varepsilon**2) - (u + 2*u1)/(30*\varepsilon**3) + (3*u + 4*u1)/(120*\varepsilon**4) - (u + u1)/(120*\varepsilon**5)

h^5 =>


\varepsilon**6*(u - u1)/144 + \varepsilon**5*(4*u - 5*u1)/144 + \varepsilon**4*(7*u - 12*u1)/144 + \varepsilon**3*(u - 4*u1)/36 - \varepsilon**2*(u + 15*u1)/144 - \varepsilon*(8*u + 7*u1)/144 - u1/18 - (8*u - 7*u1)/(144*\varepsilon) + (u - 15*u1)/(144*\varepsilon**2) + (u + 4*u1)/(36*\varepsilon**3) - (7*u + 12*u1)/(144*\varepsilon**4) + (4*u + 5*u1)/(144*\varepsilon**5) - (u + u1)/(144*\varepsilon**6)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{\varepsilon^{5} \left(u - u_{1}\right)}{120} + \frac{\varepsilon^{4} \left(3 u - 4 u_{1}\right)}{120} + \frac{\varepsilon^{3} \left(u - 2 u_{1}\right)}{30} - \frac{\varepsilon^{2} u_{1}}{15} - \frac{\varepsilon \left(u + 7 u_{1}\right)}{120} - \frac{u}{20} + \frac{u - 7 u_{1}}{120 \varepsilon} + \frac{u_{1}}{15 \varepsilon^{2}} - \frac{u + 2 u_{1}}{30 \varepsilon^{3}} + \frac{3 u + 4 u_{1}}{120 \varepsilon^{4}} - \frac{u + u_{1}}{120 \varepsilon^{5}}
\right)
+h^5\left(
\frac{\varepsilon^{6} \left(u - u_{1}\right)}{144} + \frac{\varepsilon^{5} \left(4 u - 5 u_{1}\right)}{144} + \frac{\varepsilon^{4} \left(7 u - 12 u_{1}\right)}{144} + \frac{\varepsilon^{3} \left(u - 4 u_{1}\right)}{36} - \frac{\varepsilon^{2} \left(u + 15 u_{1}\right)}{144} - \frac{\varepsilon \left(8 u + 7 u_{1}\right)}{144} - \frac{u_{1}}{18} - \frac{8 u - 7 u_{1}}

In [37]:
f1[4].subs(sol).expand().simplify()

(\varepsilon**10*exp(t/\varepsilon) + 5*\varepsilon**9*exp(t/\varepsilon) + 10*\varepsilon**8*exp(t/\varepsilon) + 10*\varepsilon**7*exp(t/\varepsilon) + 5*\varepsilon**6*exp(t/\varepsilon) + \varepsilon**5*exp(t/\varepsilon) + \varepsilon**5*exp(t*(\varepsilon + 2)) - 5*\varepsilon**4*exp(t*(\varepsilon + 2)) + 10*\varepsilon**3*exp(t*(\varepsilon + 2)) - 10*\varepsilon**2*exp(t*(\varepsilon + 2)) + 5*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(120*\varepsilon**5*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [38]:
f2[4].subs(sol).expand().simplify()

(-\varepsilon**12*exp(t/\varepsilon) - 6*\varepsilon**11*exp(t/\varepsilon) - 15*\varepsilon**10*exp(t/\varepsilon) - 20*\varepsilon**9*exp(t/\varepsilon) - 15*\varepsilon**8*exp(t/\varepsilon) - 6*\varepsilon**7*exp(t/\varepsilon) - \varepsilon**6*exp(t/\varepsilon) + \varepsilon**6*exp(t*(\varepsilon + 2)) - 6*\varepsilon**5*exp(t*(\varepsilon + 2)) + 15*\varepsilon**4*exp(t*(\varepsilon + 2)) - 20*\varepsilon**3*exp(t*(\varepsilon + 2)) + 15*\varepsilon**2*exp(t*(\varepsilon + 2)) - 6*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(120*\varepsilon**6*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [39]:
print(latex(f2[4].subs(sol).expand().simplify()))

\frac{\left(- \varepsilon^{12} e^{\frac{t}{\varepsilon}} - 6 \varepsilon^{11} e^{\frac{t}{\varepsilon}} - 15 \varepsilon^{10} e^{\frac{t}{\varepsilon}} - 20 \varepsilon^{9} e^{\frac{t}{\varepsilon}} - 15 \varepsilon^{8} e^{\frac{t}{\varepsilon}} - 6 \varepsilon^{7} e^{\frac{t}{\varepsilon}} - \varepsilon^{6} e^{\frac{t}{\varepsilon}} + \varepsilon^{6} e^{t \left(\varepsilon + 2\right)} - 6 \varepsilon^{5} e^{t \left(\varepsilon + 2\right)} + 15 \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} - 20 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} + 15 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} - 6 \varepsilon e^{t \left(\varepsilon + 2\right)} + e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{120 \varepsilon^{6} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


## Dormand–Prince method 5 order

In [40]:
set_clip(8, 7, Rational(0, 1))
r = DormandPrince5(f, Matrix([u(t), u1(t)]))

In [41]:
F1 = clip((T(u(t+h))-T(u(t)))/h - T(r[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-\varepsilon*(u - u1)/2 + u_{tt}/2 + (u + u1)/(2*\varepsilon)

h^2 =>


...

h^5 =>


-\varepsilon**5*(u - u1)/600 - \varepsilon**4*(3*u - 4*u1)/600 - \varepsilon**3*(u - 2*u1)/150 + \varepsilon**2*u1/75 + \varepsilon*(u + 7*u1)/600 + (36*u + 5*u_{tttttt})/3600 - (u - 7*u1)/(600*\varepsilon) - u1/(75*\varepsilon**2) + (u + 2*u1)/(150*\varepsilon**3) - (3*u + 4*u1)/(600*\varepsilon**4) + (u + u1)/(600*\varepsilon**5)

h^6 =>


u_{ttttttt}/5040

- u_{1} + u_{t}
+h\left(
- \frac{\varepsilon \left(u - u_{1}\right)}{2} + \frac{u_{tt}}{2} + \frac{u + u_{1}}{2 \varepsilon}
\right)
+h^2\left(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{6} - \frac{\varepsilon u_{1}}{6} - \frac{2 u_{1} - u_{ttt}}{6} + \frac{u_{1}}{6 \varepsilon} - \frac{u + u_{1}}{6 \varepsilon^{2}}
\right)
+h^3\left(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{24} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{24} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{24} + \frac{2 u + u_{tttt}}{24} + \frac{u + 3 u_{1}}{24 \varepsilon} - \frac{u + 2 u_{1}}{24 \varepsilon^{2}} + \frac{u + u_{1}}{24 \varepsilon^{3}}
\right)
+h^4\left(
\frac{\varepsilon^{4} \left(u - u_{1}\right)}{120} + \frac{\varepsilon^{3} \left(2 u - 3 u_{1}\right)}{120} + \frac{\varepsilon^{2} \left(2 u - 5 u_{1}\right)}{120} - \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{120} - \frac{4 u_{1} - u_{ttttt}}{120} - \frac{2 u - 3 u_{1}}{120 \varepsilon} - \frac{2 u + 5 u_{1}}{120 \varepsilon^{2}} + \f

In [42]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h - T(r[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


\varepsilon**2*(u - u1)/2 - \varepsilon*u1/2 - (2*u1 - u1_{tt})/2 + u1/(2*\varepsilon) - (u + u1)/(2*\varepsilon**2)

h^2 =>


...

h^5 =>


\varepsilon**6*(u - u1)/600 + \varepsilon**5*(4*u - 5*u1)/600 + \varepsilon**4*(7*u - 12*u1)/600 + \varepsilon**3*(u - 4*u1)/150 - \varepsilon**2*(u + 15*u1)/600 - \varepsilon*(8*u + 7*u1)/600 - (48*u1 - 5*u1_{tttttt})/3600 - (8*u - 7*u1)/(600*\varepsilon) + (u - 15*u1)/(600*\varepsilon**2) + (u + 4*u1)/(150*\varepsilon**3) - (7*u + 12*u1)/(600*\varepsilon**4) + (4*u + 5*u1)/(600*\varepsilon**5) - (u + u1)/(600*\varepsilon**6)

h^6 =>


u1_{ttttttt}/5040

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{2} - \frac{\varepsilon u_{1}}{2} - \frac{2 u_{1} - u1_{tt}}{2} + \frac{u_{1}}{2 \varepsilon} - \frac{u + u_{1}}{2 \varepsilon^{2}}
\right)
+h^2\left(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{6} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{6} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{6} + \frac{2 u + u1_{ttt}}{6} + \frac{u + 3 u_{1}}{6 \varepsilon} - \frac{u + 2 u_{1}}{6 \varepsilon^{2}} + \frac{u + u_{1}}{6 \varepsilon^{3}}
\right)
+h^3\left(
\frac{\varepsilon^{4} \left(u - u_{1}\right)}{24} + \frac{\varepsilon^{3} \left(2 u - 3 u_{1}\right)}{24} + \frac{\varepsilon^{2} \left(2 u - 5 u_{1}\right)}{24} - \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{24} - \frac{4 u_{1} - u1_{tttt}}{24} - \frac{2 u - 3 u_{1}}{24 \varepsilon} - \frac{2 u + 5 u_{1}}{24 \varepsilon^{2}} + \frac{2 u + 3 u_{1}}{24 \varepsilon^{3}} - \frac{u + u_{1}}{24 \varepsilo

In [43]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-\varepsilon**5*(u - u1)/3600 - \varepsilon**4*(3*u - 4*u1)/3600 - \varepsilon**3*(u - 2*u1)/900 + \varepsilon**2*u1/450 + \varepsilon*(u + 7*u1)/3600 + u/600 - (u - 7*u1)/(3600*\varepsilon) - u1/(450*\varepsilon**2) + (u + 2*u1)/(900*\varepsilon**3) - (3*u + 4*u1)/(3600*\varepsilon**4) + (u + u1)/(3600*\varepsilon**5)

h^6 =>


-\varepsilon**6*(u - u1)/2100 - \varepsilon**5*(4*u - 5*u1)/2100 - \varepsilon**4*(7*u - 12*u1)/2100 - \varepsilon**3*(u - 4*u1)/525 + \varepsilon**2*(u + 15*u1)/2100 + \varepsilon*(8*u + 7*u1)/2100 + 2*u1/525 + (8*u - 7*u1)/(2100*\varepsilon) - (u - 15*u1)/(2100*\varepsilon**2) - (u + 4*u1)/(525*\varepsilon**3) + (7*u + 12*u1)/(2100*\varepsilon**4) - (4*u + 5*u1)/(2100*\varepsilon**5) + (u + u1)/(2100*\varepsilon**6)

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{\varepsilon^{5} \left(u - u_{1}\right)}{3600} - \frac{\varepsilon^{4} \left(3 u - 4 u_{1}\right)}{3600} - \frac{\varepsilon^{3} \left(u - 2 u_{1}\right)}{900} + \frac{\varepsilon^{2} u_{1}}{450} + \frac{\varepsilon \left(u + 7 u_{1}\right)}{3600} + \frac{u}{600} - \frac{u - 7 u_{1}}{3600 \varepsilon} - \frac{u_{1}}{450 \varepsilon^{2}} + \frac{u + 2 u_{1}}{900 \varepsilon^{3}} - \frac{3 u + 4 u_{1}}{3600 \varepsilon^{4}} + \frac{u + u_{1}}{3600 \varepsilon^{5}}
\right)
+h^6\left(
- \frac{\varepsilon^{6} \left(u - u_{1}\right)}{2100} - \frac{\varepsilon^{5} \left(4 u - 5 u_{1}\right)}{2100} - \frac{\varepsilon^{4} \left(7 u - 12 u_{1}\right)}{2100} - \frac{\varepsilon^{3} \left(u - 4 u_{1}\right)}{525} + \frac{\varepsilon^{2} \left(u + 15 u_{1}\right)}{2100} + \frac{\varepsilon \left(8 u + 7 u_{1}\right)}{2100} + \frac{2 u_{1}}{525} + \frac{8 u - 7 u_{1}}{2100 \varepsilon

In [44]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^5 =>


\varepsilon**6*(u - u1)/3600 + \varepsilon**5*(4*u - 5*u1)/3600 + \varepsilon**4*(7*u - 12*u1)/3600 + \varepsilon**3*(u - 4*u1)/900 - \varepsilon**2*(u + 15*u1)/3600 - \varepsilon*(8*u + 7*u1)/3600 - u1/450 - (8*u - 7*u1)/(3600*\varepsilon) + (u - 15*u1)/(3600*\varepsilon**2) + (u + 4*u1)/(900*\varepsilon**3) - (7*u + 12*u1)/(3600*\varepsilon**4) + (4*u + 5*u1)/(3600*\varepsilon**5) - (u + u1)/(3600*\varepsilon**6)

h^6 =>


\varepsilon**7*(u - u1)/2100 + \varepsilon**6*(5*u - 6*u1)/2100 + \varepsilon**5*(11*u - 17*u1)/2100 + \varepsilon**4*(11*u - 28*u1)/2100 + \varepsilon**3*(3*u - 31*u1)/2100 - \varepsilon**2*(9*u + 22*u1)/2100 - \varepsilon*(7*u + 15*u1)/2100 - u/150 + (7*u - 15*u1)/(2100*\varepsilon) - (9*u - 22*u1)/(2100*\varepsilon**2) - (3*u + 31*u1)/(2100*\varepsilon**3) + (11*u + 28*u1)/(2100*\varepsilon**4) - (11*u + 17*u1)/(2100*\varepsilon**5) + (5*u + 6*u1)/(2100*\varepsilon**6) - (u + u1)/(2100*\varepsilon**7)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{\varepsilon^{6} \left(u - u_{1}\right)}{3600} + \frac{\varepsilon^{5} \left(4 u - 5 u_{1}\right)}{3600} + \frac{\varepsilon^{4} \left(7 u - 12 u_{1}\right)}{3600} + \frac{\varepsilon^{3} \left(u - 4 u_{1}\right)}{900} - \frac{\varepsilon^{2} \left(u + 15 u_{1}\right)}{3600} - \frac{\varepsilon \left(8 u + 7 u_{1}\right)}{3600} - \frac{u_{1}}{450} - \frac{8 u - 7 u_{1}}{3600 \varepsilon} + \frac{u - 15 u_{1}}{3600 \varepsilon^{2}} + \frac{u + 4 u_{1}}{900 \varepsilon^{3}} - \frac{7 u + 12 u_{1}}{3600 \varepsilon^{4}} + \frac{4 u + 5 u_{1}}{3600 \varepsilon^{5}} - \frac{u + u_{1}}{3600 \varepsilon^{6}}
\right)
+h^6\left(
\frac{\varepsilon^{7} \left(u - u_{1}\right)}{2100} + \frac{\varepsilon^{6} \left(5 u - 6 u_{1}\right)}{2100} + \frac{\varepsilon^{5} \left(11 u - 17 u_{1}\right)}{2100} + \frac{\varepsilon^{4} \l

In [45]:
f1[5].subs(sol).expand().simplify()

(\varepsilon**12*exp(t/\varepsilon) + 6*\varepsilon**11*exp(t/\varepsilon) + 15*\varepsilon**10*exp(t/\varepsilon) + 20*\varepsilon**9*exp(t/\varepsilon) + 15*\varepsilon**8*exp(t/\varepsilon) + 6*\varepsilon**7*exp(t/\varepsilon) + \varepsilon**6*exp(t/\varepsilon) - \varepsilon**6*exp(t*(\varepsilon + 2)) + 6*\varepsilon**5*exp(t*(\varepsilon + 2)) - 15*\varepsilon**4*exp(t*(\varepsilon + 2)) + 20*\varepsilon**3*exp(t*(\varepsilon + 2)) - 15*\varepsilon**2*exp(t*(\varepsilon + 2)) + 6*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(3600*\varepsilon**6*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [46]:
f2[5].subs(sol).expand().simplify()

(-\varepsilon**14*exp(t/\varepsilon) - 7*\varepsilon**13*exp(t/\varepsilon) - 21*\varepsilon**12*exp(t/\varepsilon) - 35*\varepsilon**11*exp(t/\varepsilon) - 35*\varepsilon**10*exp(t/\varepsilon) - 21*\varepsilon**9*exp(t/\varepsilon) - 7*\varepsilon**8*exp(t/\varepsilon) - \varepsilon**7*exp(t/\varepsilon) - \varepsilon**7*exp(t*(\varepsilon + 2)) + 7*\varepsilon**6*exp(t*(\varepsilon + 2)) - 21*\varepsilon**5*exp(t*(\varepsilon + 2)) + 35*\varepsilon**4*exp(t*(\varepsilon + 2)) - 35*\varepsilon**3*exp(t*(\varepsilon + 2)) + 21*\varepsilon**2*exp(t*(\varepsilon + 2)) - 7*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(3600*\varepsilon**7*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [47]:
print(latex(f2[5].subs(sol).expand().simplify()))

\frac{\left(- \varepsilon^{14} e^{\frac{t}{\varepsilon}} - 7 \varepsilon^{13} e^{\frac{t}{\varepsilon}} - 21 \varepsilon^{12} e^{\frac{t}{\varepsilon}} - 35 \varepsilon^{11} e^{\frac{t}{\varepsilon}} - 35 \varepsilon^{10} e^{\frac{t}{\varepsilon}} - 21 \varepsilon^{9} e^{\frac{t}{\varepsilon}} - 7 \varepsilon^{8} e^{\frac{t}{\varepsilon}} - \varepsilon^{7} e^{\frac{t}{\varepsilon}} - \varepsilon^{7} e^{t \left(\varepsilon + 2\right)} + 7 \varepsilon^{6} e^{t \left(\varepsilon + 2\right)} - 21 \varepsilon^{5} e^{t \left(\varepsilon + 2\right)} + 35 \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} - 35 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} + 21 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} - 7 \varepsilon e^{t \left(\varepsilon + 2\right)} + e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{3600 \varepsilon^{7} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


## Dormand–Prince method 4 order

In [48]:
set_clip(7, 6, Rational(0, 1))
r = DormandPrince4(f, Matrix([u(t), u1(t)]))

In [49]:
F1 = clip((T(u(t+h))-T(u(t)))/h - T(r[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-\varepsilon*(u - u1)/2 + u_{tt}/2 + (u + u1)/(2*\varepsilon)

h^2 =>


...

h^4 =>


1097*\varepsilon**4*(u - u1)/120000 + 1097*\varepsilon**3*(2*u - 3*u1)/120000 + 1097*\varepsilon**2*(2*u - 5*u1)/120000 - 1097*\varepsilon*(2*u + 3*u1)/120000 - (1097*u1 - 250*u_{ttttt})/30000 - 1097*(2*u - 3*u1)/(120000*\varepsilon) - 1097*(2*u + 5*u1)/(120000*\varepsilon**2) + 1097*(2*u + 3*u1)/(120000*\varepsilon**3) - 1097*(u + u1)/(120000*\varepsilon**4)

h^5 =>


-161*\varepsilon**5*(u - u1)/120000 - 161*\varepsilon**4*(3*u - 4*u1)/120000 - 161*\varepsilon**3*(u - 2*u1)/30000 + 161*\varepsilon**2*u1/15000 + 161*\varepsilon*(u + 7*u1)/120000 + (1449*u + 250*u_{tttttt})/180000 - 161*(u - 7*u1)/(120000*\varepsilon) - 161*u1/(15000*\varepsilon**2) + 161*(u + 2*u1)/(30000*\varepsilon**3) - 161*(3*u + 4*u1)/(120000*\varepsilon**4) + 161*(u + u1)/(120000*\varepsilon**5)

- u_{1} + u_{t}
+h\left(
- \frac{\varepsilon \left(u - u_{1}\right)}{2} + \frac{u_{tt}}{2} + \frac{u + u_{1}}{2 \varepsilon}
\right)
+h^2\left(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{6} - \frac{\varepsilon u_{1}}{6} - \frac{2 u_{1} - u_{ttt}}{6} + \frac{u_{1}}{6 \varepsilon} - \frac{u + u_{1}}{6 \varepsilon^{2}}
\right)
+h^3\left(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{24} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{24} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{24} + \frac{2 u + u_{tttt}}{24} + \frac{u + 3 u_{1}}{24 \varepsilon} - \frac{u + 2 u_{1}}{24 \varepsilon^{2}} + \frac{u + u_{1}}{24 \varepsilon^{3}}
\right)
+h^4\left(
\frac{1097 \varepsilon^{4} \left(u - u_{1}\right)}{120000} + \frac{1097 \varepsilon^{3} \left(2 u - 3 u_{1}\right)}{120000} + \frac{1097 \varepsilon^{2} \left(2 u - 5 u_{1}\right)}{120000} - \frac{1097 \varepsilon \left(2 u + 3 u_{1}\right)}{120000} - \frac{1097 u_{1} - 250 u_{ttttt}}{30000} - \frac{1097 \left(2 u - 3 u_{1}\right)}{120000

In [50]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h - T(r[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


\varepsilon**2*(u - u1)/2 - \varepsilon*u1/2 - (2*u1 - u1_{tt})/2 + u1/(2*\varepsilon) - (u + u1)/(2*\varepsilon**2)

h^2 =>


...

h^4 =>


-1097*\varepsilon**5*(u - u1)/120000 - 1097*\varepsilon**4*(3*u - 4*u1)/120000 - 1097*\varepsilon**3*(u - 2*u1)/30000 + 1097*\varepsilon**2*u1/15000 + 1097*\varepsilon*(u + 7*u1)/120000 + (3291*u + 500*u1_{ttttt})/60000 - 1097*(u - 7*u1)/(120000*\varepsilon) - 1097*u1/(15000*\varepsilon**2) + 1097*(u + 2*u1)/(30000*\varepsilon**3) - 1097*(3*u + 4*u1)/(120000*\varepsilon**4) + 1097*(u + u1)/(120000*\varepsilon**5)

h^5 =>


161*\varepsilon**6*(u - u1)/120000 + 161*\varepsilon**5*(4*u - 5*u1)/120000 + 161*\varepsilon**4*(7*u - 12*u1)/120000 + 161*\varepsilon**3*(u - 4*u1)/30000 - 161*\varepsilon**2*(u + 15*u1)/120000 - 161*\varepsilon*(8*u + 7*u1)/120000 - (966*u1 - 125*u1_{tttttt})/90000 - 161*(8*u - 7*u1)/(120000*\varepsilon) + 161*(u - 15*u1)/(120000*\varepsilon**2) + 161*(u + 4*u1)/(30000*\varepsilon**3) - 161*(7*u + 12*u1)/(120000*\varepsilon**4) + 161*(4*u + 5*u1)/(120000*\varepsilon**5) - 161*(u + u1)/(120000*\varepsilon**6)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{2} - \frac{\varepsilon u_{1}}{2} - \frac{2 u_{1} - u1_{tt}}{2} + \frac{u_{1}}{2 \varepsilon} - \frac{u + u_{1}}{2 \varepsilon^{2}}
\right)
+h^2\left(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{6} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{6} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{6} + \frac{2 u + u1_{ttt}}{6} + \frac{u + 3 u_{1}}{6 \varepsilon} - \frac{u + 2 u_{1}}{6 \varepsilon^{2}} + \frac{u + u_{1}}{6 \varepsilon^{3}}
\right)
+h^3\left(
\frac{\varepsilon^{4} \left(u - u_{1}\right)}{24} + \frac{\varepsilon^{3} \left(2 u - 3 u_{1}\right)}{24} + \frac{\varepsilon^{2} \left(2 u - 5 u_{1}\right)}{24} - \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{24} - \frac{4 u_{1} - u1_{tttt}}{24} - \frac{2 u - 3 u_{1}}{24 \varepsilon} - \frac{2 u + 5 u_{1}}{24 \varepsilon^{2}} + \frac{2 u + 3 u_{1}}{24 \varepsilon^{3}} - \frac{u + u_{1}}{24 \varepsilo

In [51]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


97*\varepsilon**4*(u - u1)/120000 + 97*\varepsilon**3*(2*u - 3*u1)/120000 + 97*\varepsilon**2*(2*u - 5*u1)/120000 - 97*\varepsilon*(2*u + 3*u1)/120000 - 97*u1/30000 - 97*(2*u - 3*u1)/(120000*\varepsilon) - 97*(2*u + 5*u1)/(120000*\varepsilon**2) + 97*(2*u + 3*u1)/(120000*\varepsilon**3) - 97*(u + u1)/(120000*\varepsilon**4)

h^5 =>


77*\varepsilon**5*(u - u1)/90000 + 77*\varepsilon**4*(3*u - 4*u1)/90000 + 77*\varepsilon**3*(u - 2*u1)/22500 - 77*\varepsilon**2*u1/11250 - 77*\varepsilon*(u + 7*u1)/90000 - 77*u/15000 + 77*(u - 7*u1)/(90000*\varepsilon) + 77*u1/(11250*\varepsilon**2) - 77*(u + 2*u1)/(22500*\varepsilon**3) + 77*(3*u + 4*u1)/(90000*\varepsilon**4) - 77*(u + u1)/(90000*\varepsilon**5)

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{97 \varepsilon^{4} \left(u - u_{1}\right)}{120000} + \frac{97 \varepsilon^{3} \left(2 u - 3 u_{1}\right)}{120000} + \frac{97 \varepsilon^{2} \left(2 u - 5 u_{1}\right)}{120000} - \frac{97 \varepsilon \left(2 u + 3 u_{1}\right)}{120000} - \frac{97 u_{1}}{30000} - \frac{97 \left(2 u - 3 u_{1}\right)}{120000 \varepsilon} - \frac{97 \left(2 u + 5 u_{1}\right)}{120000 \varepsilon^{2}} + \frac{97 \left(2 u + 3 u_{1}\right)}{120000 \varepsilon^{3}} - \frac{97 \left(u + u_{1}\right)}{120000 \varepsilon^{4}}
\right)
+h^5\left(
\frac{77 \varepsilon^{5} \left(u - u_{1}\right)}{90000} + \frac{77 \varepsilon^{4} \left(3 u - 4 u_{1}\right)}{90000} + \frac{77 \varepsilon^{3} \left(u - 2 u_{1}\right)}{22500} - \frac{77 \varepsilon^{2} u_{1}}{11250} - \frac{77 \varepsilon \left(u + 7 u_{1}\right)}{90000} - \frac{77 u}{15000} + \frac{77 \left(u - 7 u_{1}\right)}{90000 \varepsilon} + \frac{77 u_{1}}{11250 \varep

In [52]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^4 =>


-97*\varepsilon**5*(u - u1)/120000 - 97*\varepsilon**4*(3*u - 4*u1)/120000 - 97*\varepsilon**3*(u - 2*u1)/30000 + 97*\varepsilon**2*u1/15000 + 97*\varepsilon*(u + 7*u1)/120000 + 97*u/20000 - 97*(u - 7*u1)/(120000*\varepsilon) - 97*u1/(15000*\varepsilon**2) + 97*(u + 2*u1)/(30000*\varepsilon**3) - 97*(3*u + 4*u1)/(120000*\varepsilon**4) + 97*(u + u1)/(120000*\varepsilon**5)

h^5 =>


-77*\varepsilon**6*(u - u1)/90000 - 77*\varepsilon**5*(4*u - 5*u1)/90000 - 77*\varepsilon**4*(7*u - 12*u1)/90000 - 77*\varepsilon**3*(u - 4*u1)/22500 + 77*\varepsilon**2*(u + 15*u1)/90000 + 77*\varepsilon*(8*u + 7*u1)/90000 + 77*u1/11250 + 77*(8*u - 7*u1)/(90000*\varepsilon) - 77*(u - 15*u1)/(90000*\varepsilon**2) - 77*(u + 4*u1)/(22500*\varepsilon**3) + 77*(7*u + 12*u1)/(90000*\varepsilon**4) - 77*(4*u + 5*u1)/(90000*\varepsilon**5) + 77*(u + u1)/(90000*\varepsilon**6)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{97 \varepsilon^{5} \left(u - u_{1}\right)}{120000} - \frac{97 \varepsilon^{4} \left(3 u - 4 u_{1}\right)}{120000} - \frac{97 \varepsilon^{3} \left(u - 2 u_{1}\right)}{30000} + \frac{97 \varepsilon^{2} u_{1}}{15000} + \frac{97 \varepsilon \left(u + 7 u_{1}\right)}{120000} + \frac{97 u}{20000} - \frac{97 \left(u - 7 u_{1}\right)}{120000 \varepsilon} - \frac{97 u_{1}}{15000 \varepsilon^{2}} + \frac{97 \left(u + 2 u_{1}\right)}{30000 \varepsilon^{3}} - \frac{97 \left(3 u + 4 u_{1}\right)}{120000 \varepsilon^{4}} + \frac{97 \left(u + u_{1}\right)}{120000 \varepsilon^{5}}
\right)
+h^5\left(
- \frac{77 \varepsilon^{6} \left(u - u_{1}\right)}{90000} - \frac{77 \varepsilon^{5} \left(4 u - 5 u_{1}\right)}{90000} - \frac{77 \varepsilon^{4} \left(7 u - 12 u_{1}\right)}{90000} - \frac{77 \varepsilon^{3} \left(u - 4 u_{1}\right)}{22500} + \frac

In [53]:
f1[4].subs(sol).expand().simplify()

97*(-\varepsilon**10*exp(t/\varepsilon) - 5*\varepsilon**9*exp(t/\varepsilon) - 10*\varepsilon**8*exp(t/\varepsilon) - 10*\varepsilon**7*exp(t/\varepsilon) - 5*\varepsilon**6*exp(t/\varepsilon) - \varepsilon**5*exp(t/\varepsilon) - \varepsilon**5*exp(t*(\varepsilon + 2)) + 5*\varepsilon**4*exp(t*(\varepsilon + 2)) - 10*\varepsilon**3*exp(t*(\varepsilon + 2)) + 10*\varepsilon**2*exp(t*(\varepsilon + 2)) - 5*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(120000*\varepsilon**5*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [54]:
f2[4].subs(sol).expand().simplify()

97*(\varepsilon**12*exp(t/\varepsilon) + 6*\varepsilon**11*exp(t/\varepsilon) + 15*\varepsilon**10*exp(t/\varepsilon) + 20*\varepsilon**9*exp(t/\varepsilon) + 15*\varepsilon**8*exp(t/\varepsilon) + 6*\varepsilon**7*exp(t/\varepsilon) + \varepsilon**6*exp(t/\varepsilon) - \varepsilon**6*exp(t*(\varepsilon + 2)) + 6*\varepsilon**5*exp(t*(\varepsilon + 2)) - 15*\varepsilon**4*exp(t*(\varepsilon + 2)) + 20*\varepsilon**3*exp(t*(\varepsilon + 2)) - 15*\varepsilon**2*exp(t*(\varepsilon + 2)) + 6*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(120000*\varepsilon**6*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [55]:
print(latex(f2[4].subs(sol).expand().simplify()))

\frac{97 \left(\varepsilon^{12} e^{\frac{t}{\varepsilon}} + 6 \varepsilon^{11} e^{\frac{t}{\varepsilon}} + 15 \varepsilon^{10} e^{\frac{t}{\varepsilon}} + 20 \varepsilon^{9} e^{\frac{t}{\varepsilon}} + 15 \varepsilon^{8} e^{\frac{t}{\varepsilon}} + 6 \varepsilon^{7} e^{\frac{t}{\varepsilon}} + \varepsilon^{6} e^{\frac{t}{\varepsilon}} - \varepsilon^{6} e^{t \left(\varepsilon + 2\right)} + 6 \varepsilon^{5} e^{t \left(\varepsilon + 2\right)} - 15 \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} + 20 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} - 15 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} + 6 \varepsilon e^{t \left(\varepsilon + 2\right)} - e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{120000 \varepsilon^{6} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


### Adams–Bashforth method 4 order

In [56]:
set_clip(7, 6, Rational(0, 1))
m = AdamsBashforth4(g, Matrix([u(t), u1(t)]))

In [57]:
F1 = clip((T(u(t+4*h))-T(u(t+3*h)))/h - T(m[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-7*(u1_{t} - u_{tt})/2

h^2 =>


...

h^4 =>


-(4435*u1_{tttt} - 4686*u_{ttttt})/720

h^5 =>


-(5757*u1_{ttttt} - 6734*u_{tttttt})/1440

- u_{1} + u_{t}
+h\left(
- \frac{7 \left(u1_{t} - u_{tt}\right)}{2}
\right)
+h^2\left(
- \frac{37 \left(u1_{tt} - u_{ttt}\right)}{6}
\right)
+h^3\left(
- \frac{175 \left(u1_{ttt} - u_{tttt}\right)}{24}
\right)
+h^4\left(
- \frac{4435 u1_{tttt} - 4686 u_{ttttt}}{720}
\right)
+h^5\left(
- \frac{5757 u1_{ttttt} - 6734 u_{tttttt}}{1440}
\right)


In [58]:
F2 = clip((T(u1(t+4*h))-T(u1(t+3*h)))/h - T(m[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


7*\varepsilon*(u1_{t} - u_{t})/2 + 7*u1_{tt}/2 + 7*(u1_{t} + u_{t})/(2*\varepsilon)

h^2 =>


...

h^4 =>


887*\varepsilon*(u1_{tttt} - u_{tttt})/144 + 781*u1_{ttttt}/120 + 887*(u1_{tttt} + u_{tttt})/(144*\varepsilon)

h^5 =>


1919*\varepsilon*(u1_{ttttt} - u_{ttttt})/480 + 3367*u1_{tttttt}/720 + 1919*(u1_{ttttt} + u_{ttttt})/(480*\varepsilon)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{7 \varepsilon \left(u1_{t} - u_{t}\right)}{2} + \frac{7 u1_{tt}}{2} + \frac{7 \left(u1_{t} + u_{t}\right)}{2 \varepsilon}
\right)
+h^2\left(
\frac{37 \varepsilon \left(u1_{tt} - u_{tt}\right)}{6} + \frac{37 u1_{ttt}}{6} + \frac{37 \left(u1_{tt} + u_{tt}\right)}{6 \varepsilon}
\right)
+h^3\left(
\frac{175 \varepsilon \left(u1_{ttt} - u_{ttt}\right)}{24} + \frac{175 u1_{tttt}}{24} + \frac{175 \left(u1_{ttt} + u_{ttt}\right)}{24 \varepsilon}
\right)
+h^4\left(
\frac{887 \varepsilon \left(u1_{tttt} - u_{tttt}\right)}{144} + \frac{781 u1_{ttttt}}{120} + \frac{887 \left(u1_{tttt} + u_{tttt}\right)}{144 \varepsilon}
\right)
+h^5\left(
\frac{1919 \varepsilon \left(u1_{ttttt} - u_{ttttt}\right)}{480} + \frac{3367 u1_{tttttt}}{720} + \frac{1919 \left(u1_{ttttt} + u_{ttttt}\right)}{480 \varepsilon}
\right)


In [59]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-251*\varepsilon**4*(u - u1)/720 - 251*\varepsilon**3*(2*u - 3*u1)/720 - 251*\varepsilon**2*(2*u - 5*u1)/720 + 251*\varepsilon*(2*u + 3*u1)/720 + 251*u1/180 + 251*(2*u - 3*u1)/(720*\varepsilon) + 251*(2*u + 5*u1)/(720*\varepsilon**2) - 251*(2*u + 3*u1)/(720*\varepsilon**3) + 251*(u + u1)/(720*\varepsilon**4)

h^5 =>


-13*\varepsilon**5*(u - u1)/24 - 13*\varepsilon**4*(3*u - 4*u1)/24 - 13*\varepsilon**3*(u - 2*u1)/6 + 13*\varepsilon**2*u1/3 + 13*\varepsilon*(u + 7*u1)/24 + 13*u/4 - 13*(u - 7*u1)/(24*\varepsilon) - 13*u1/(3*\varepsilon**2) + 13*(u + 2*u1)/(6*\varepsilon**3) - 13*(3*u + 4*u1)/(24*\varepsilon**4) + 13*(u + u1)/(24*\varepsilon**5)

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{251 \varepsilon^{4} \left(u - u_{1}\right)}{720} - \frac{251 \varepsilon^{3} \left(2 u - 3 u_{1}\right)}{720} - \frac{251 \varepsilon^{2} \left(2 u - 5 u_{1}\right)}{720} + \frac{251 \varepsilon \left(2 u + 3 u_{1}\right)}{720} + \frac{251 u_{1}}{180} + \frac{251 \left(2 u - 3 u_{1}\right)}{720 \varepsilon} + \frac{251 \left(2 u + 5 u_{1}\right)}{720 \varepsilon^{2}} - \frac{251 \left(2 u + 3 u_{1}\right)}{720 \varepsilon^{3}} + \frac{251 \left(u + u_{1}\right)}{720 \varepsilon^{4}}
\right)
+h^5\left(
- \frac{13 \varepsilon^{5} \left(u - u_{1}\right)}{24} - \frac{13 \varepsilon^{4} \left(3 u - 4 u_{1}\right)}{24} - \frac{13 \varepsilon^{3} \left(u - 2 u_{1}\right)}{6} + \frac{13 \varepsilon^{2} u_{1}}{3} + \frac{13 \varepsilon \left(u + 7 u_{1}\right)}{24} + \frac{13 u}{4} - \frac{13 \left(u - 7 u_{1}\right)}{24 \varepsilon} - \frac{13 u_{1}}{3 \varepsilon^{2}} + \frac{13 \left(u + 2 u_{1}\r

In [60]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^4 =>


251*\varepsilon**5*(u - u1)/720 + 251*\varepsilon**4*(3*u - 4*u1)/720 + 251*\varepsilon**3*(u - 2*u1)/180 - 251*\varepsilon**2*u1/90 - 251*\varepsilon*(u + 7*u1)/720 - 251*u/120 + 251*(u - 7*u1)/(720*\varepsilon) + 251*u1/(90*\varepsilon**2) - 251*(u + 2*u1)/(180*\varepsilon**3) + 251*(3*u + 4*u1)/(720*\varepsilon**4) - 251*(u + u1)/(720*\varepsilon**5)

h^5 =>


13*\varepsilon**6*(u - u1)/24 + 13*\varepsilon**5*(4*u - 5*u1)/24 + 13*\varepsilon**4*(7*u - 12*u1)/24 + 13*\varepsilon**3*(u - 4*u1)/6 - 13*\varepsilon**2*(u + 15*u1)/24 - 13*\varepsilon*(8*u + 7*u1)/24 - 13*u1/3 - 13*(8*u - 7*u1)/(24*\varepsilon) + 13*(u - 15*u1)/(24*\varepsilon**2) + 13*(u + 4*u1)/(6*\varepsilon**3) - 13*(7*u + 12*u1)/(24*\varepsilon**4) + 13*(4*u + 5*u1)/(24*\varepsilon**5) - 13*(u + u1)/(24*\varepsilon**6)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{251 \varepsilon^{5} \left(u - u_{1}\right)}{720} + \frac{251 \varepsilon^{4} \left(3 u - 4 u_{1}\right)}{720} + \frac{251 \varepsilon^{3} \left(u - 2 u_{1}\right)}{180} - \frac{251 \varepsilon^{2} u_{1}}{90} - \frac{251 \varepsilon \left(u + 7 u_{1}\right)}{720} - \frac{251 u}{120} + \frac{251 \left(u - 7 u_{1}\right)}{720 \varepsilon} + \frac{251 u_{1}}{90 \varepsilon^{2}} - \frac{251 \left(u + 2 u_{1}\right)}{180 \varepsilon^{3}} + \frac{251 \left(3 u + 4 u_{1}\right)}{720 \varepsilon^{4}} - \frac{251 \left(u + u_{1}\right)}{720 \varepsilon^{5}}
\right)
+h^5\left(
\frac{13 \varepsilon^{6} \left(u - u_{1}\right)}{24} + \frac{13 \varepsilon^{5} \left(4 u - 5 u_{1}\right)}{24} + \frac{13 \varepsilon^{4} \left(7 u - 12 u_{1}\right)}{24} + \frac{13 \varepsilon^{3} \left(u - 4 u_{1}\right)}{6} - \frac{13 \varepsilon^{2} \left(u + 15 u_{

In [61]:
f1[4].subs(sol).expand().simplify()

251*(\varepsilon**10*exp(t/\varepsilon) + 5*\varepsilon**9*exp(t/\varepsilon) + 10*\varepsilon**8*exp(t/\varepsilon) + 10*\varepsilon**7*exp(t/\varepsilon) + 5*\varepsilon**6*exp(t/\varepsilon) + \varepsilon**5*exp(t/\varepsilon) + \varepsilon**5*exp(t*(\varepsilon + 2)) - 5*\varepsilon**4*exp(t*(\varepsilon + 2)) + 10*\varepsilon**3*exp(t*(\varepsilon + 2)) - 10*\varepsilon**2*exp(t*(\varepsilon + 2)) + 5*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(720*\varepsilon**5*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [62]:
f2[4].subs(sol).expand().simplify()

251*(-\varepsilon**12*exp(t/\varepsilon) - 6*\varepsilon**11*exp(t/\varepsilon) - 15*\varepsilon**10*exp(t/\varepsilon) - 20*\varepsilon**9*exp(t/\varepsilon) - 15*\varepsilon**8*exp(t/\varepsilon) - 6*\varepsilon**7*exp(t/\varepsilon) - \varepsilon**6*exp(t/\varepsilon) + \varepsilon**6*exp(t*(\varepsilon + 2)) - 6*\varepsilon**5*exp(t*(\varepsilon + 2)) + 15*\varepsilon**4*exp(t*(\varepsilon + 2)) - 20*\varepsilon**3*exp(t*(\varepsilon + 2)) + 15*\varepsilon**2*exp(t*(\varepsilon + 2)) - 6*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(720*\varepsilon**6*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [63]:
print(latex(f2[4].subs(sol).expand().simplify()))

\frac{251 \left(- \varepsilon^{12} e^{\frac{t}{\varepsilon}} - 6 \varepsilon^{11} e^{\frac{t}{\varepsilon}} - 15 \varepsilon^{10} e^{\frac{t}{\varepsilon}} - 20 \varepsilon^{9} e^{\frac{t}{\varepsilon}} - 15 \varepsilon^{8} e^{\frac{t}{\varepsilon}} - 6 \varepsilon^{7} e^{\frac{t}{\varepsilon}} - \varepsilon^{6} e^{\frac{t}{\varepsilon}} + \varepsilon^{6} e^{t \left(\varepsilon + 2\right)} - 6 \varepsilon^{5} e^{t \left(\varepsilon + 2\right)} + 15 \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} - 20 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} + 15 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} - 6 \varepsilon e^{t \left(\varepsilon + 2\right)} + e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{720 \varepsilon^{6} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


### Adams–Moulton method 4 order

In [64]:
set_clip(7, 6, Rational(0, 1))
m = AdamsMoulton4(g, Matrix([u(t), u1(t)]))

In [65]:
F1 = clip((T(u(t+3*h))-T(u(t+2*h)))/h - T(m[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-5*(u1_{t} - u_{tt})/2

h^2 =>


...

h^4 =>


-(1285*u1_{tttt} - 1266*u_{ttttt})/720

h^5 =>


-(279*u1_{ttttt} - 266*u_{tttttt})/288

- u_{1} + u_{t}
+h\left(
- \frac{5 \left(u1_{t} - u_{tt}\right)}{2}
\right)
+h^2\left(
- \frac{19 \left(u1_{tt} - u_{ttt}\right)}{6}
\right)
+h^3\left(
- \frac{65 \left(u1_{ttt} - u_{tttt}\right)}{24}
\right)
+h^4\left(
- \frac{1285 u1_{tttt} - 1266 u_{ttttt}}{720}
\right)
+h^5\left(
- \frac{279 u1_{ttttt} - 266 u_{tttttt}}{288}
\right)


In [66]:
F2 = clip((T(u1(t+3*h))-T(u1(t+2*h)))/h - T(m[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


5*\varepsilon*(u1_{t} - u_{t})/2 + 5*u1_{tt}/2 + 5*(u1_{t} + u_{t})/(2*\varepsilon)

h^2 =>


...

h^4 =>


257*\varepsilon*(u1_{tttt} - u_{tttt})/144 + 211*u1_{ttttt}/120 + 257*(u1_{tttt} + u_{tttt})/(144*\varepsilon)

h^5 =>


31*\varepsilon*(u1_{ttttt} - u_{ttttt})/32 + 133*u1_{tttttt}/144 + 31*(u1_{ttttt} + u_{ttttt})/(32*\varepsilon)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{5 \varepsilon \left(u1_{t} - u_{t}\right)}{2} + \frac{5 u1_{tt}}{2} + \frac{5 \left(u1_{t} + u_{t}\right)}{2 \varepsilon}
\right)
+h^2\left(
\frac{19 \varepsilon \left(u1_{tt} - u_{tt}\right)}{6} + \frac{19 u1_{ttt}}{6} + \frac{19 \left(u1_{tt} + u_{tt}\right)}{6 \varepsilon}
\right)
+h^3\left(
\frac{65 \varepsilon \left(u1_{ttt} - u_{ttt}\right)}{24} + \frac{65 u1_{tttt}}{24} + \frac{65 \left(u1_{ttt} + u_{ttt}\right)}{24 \varepsilon}
\right)
+h^4\left(
\frac{257 \varepsilon \left(u1_{tttt} - u_{tttt}\right)}{144} + \frac{211 u1_{ttttt}}{120} + \frac{257 \left(u1_{tttt} + u_{tttt}\right)}{144 \varepsilon}
\right)
+h^5\left(
\frac{31 \varepsilon \left(u1_{ttttt} - u_{ttttt}\right)}{32} + \frac{133 u1_{tttttt}}{144} + \frac{31 \left(u1_{ttttt} + u_{ttttt}\right)}{32 \varepsilon}
\right)


In [67]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


19*\varepsilon**4*(u - u1)/720 + 19*\varepsilon**3*(2*u - 3*u1)/720 + 19*\varepsilon**2*(2*u - 5*u1)/720 - 19*\varepsilon*(2*u + 3*u1)/720 - 19*u1/180 - 19*(2*u - 3*u1)/(720*\varepsilon) - 19*(2*u + 5*u1)/(720*\varepsilon**2) + 19*(2*u + 3*u1)/(720*\varepsilon**3) - 19*(u + u1)/(720*\varepsilon**4)

h^5 =>


\varepsilon**5*(u - u1)/48 + \varepsilon**4*(3*u - 4*u1)/48 + \varepsilon**3*(u - 2*u1)/12 - \varepsilon**2*u1/6 - \varepsilon*(u + 7*u1)/48 - u/8 + (u - 7*u1)/(48*\varepsilon) + u1/(6*\varepsilon**2) - (u + 2*u1)/(12*\varepsilon**3) + (3*u + 4*u1)/(48*\varepsilon**4) - (u + u1)/(48*\varepsilon**5)

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{19 \varepsilon^{4} \left(u - u_{1}\right)}{720} + \frac{19 \varepsilon^{3} \left(2 u - 3 u_{1}\right)}{720} + \frac{19 \varepsilon^{2} \left(2 u - 5 u_{1}\right)}{720} - \frac{19 \varepsilon \left(2 u + 3 u_{1}\right)}{720} - \frac{19 u_{1}}{180} - \frac{19 \left(2 u - 3 u_{1}\right)}{720 \varepsilon} - \frac{19 \left(2 u + 5 u_{1}\right)}{720 \varepsilon^{2}} + \frac{19 \left(2 u + 3 u_{1}\right)}{720 \varepsilon^{3}} - \frac{19 \left(u + u_{1}\right)}{720 \varepsilon^{4}}
\right)
+h^5\left(
\frac{\varepsilon^{5} \left(u - u_{1}\right)}{48} + \frac{\varepsilon^{4} \left(3 u - 4 u_{1}\right)}{48} + \frac{\varepsilon^{3} \left(u - 2 u_{1}\right)}{12} - \frac{\varepsilon^{2} u_{1}}{6} - \frac{\varepsilon \left(u + 7 u_{1}\right)}{48} - \frac{u}{8} + \frac{u - 7 u_{1}}{48 \varepsilon} + \frac{u_{1}}{6 \varepsilon^{2}} - \frac{u + 2 u_{1}}{12 \varepsilon^{3}} + \frac{3 u + 4 u_{1}}{48 \varepsilon^

In [68]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^4 =>


-19*\varepsilon**5*(u - u1)/720 - 19*\varepsilon**4*(3*u - 4*u1)/720 - 19*\varepsilon**3*(u - 2*u1)/180 + 19*\varepsilon**2*u1/90 + 19*\varepsilon*(u + 7*u1)/720 + 19*u/120 - 19*(u - 7*u1)/(720*\varepsilon) - 19*u1/(90*\varepsilon**2) + 19*(u + 2*u1)/(180*\varepsilon**3) - 19*(3*u + 4*u1)/(720*\varepsilon**4) + 19*(u + u1)/(720*\varepsilon**5)

h^5 =>


-\varepsilon**6*(u - u1)/48 - \varepsilon**5*(4*u - 5*u1)/48 - \varepsilon**4*(7*u - 12*u1)/48 - \varepsilon**3*(u - 4*u1)/12 + \varepsilon**2*(u + 15*u1)/48 + \varepsilon*(8*u + 7*u1)/48 + u1/6 + (8*u - 7*u1)/(48*\varepsilon) - (u - 15*u1)/(48*\varepsilon**2) - (u + 4*u1)/(12*\varepsilon**3) + (7*u + 12*u1)/(48*\varepsilon**4) - (4*u + 5*u1)/(48*\varepsilon**5) + (u + u1)/(48*\varepsilon**6)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{19 \varepsilon^{5} \left(u - u_{1}\right)}{720} - \frac{19 \varepsilon^{4} \left(3 u - 4 u_{1}\right)}{720} - \frac{19 \varepsilon^{3} \left(u - 2 u_{1}\right)}{180} + \frac{19 \varepsilon^{2} u_{1}}{90} + \frac{19 \varepsilon \left(u + 7 u_{1}\right)}{720} + \frac{19 u}{120} - \frac{19 \left(u - 7 u_{1}\right)}{720 \varepsilon} - \frac{19 u_{1}}{90 \varepsilon^{2}} + \frac{19 \left(u + 2 u_{1}\right)}{180 \varepsilon^{3}} - \frac{19 \left(3 u + 4 u_{1}\right)}{720 \varepsilon^{4}} + \frac{19 \left(u + u_{1}\right)}{720 \varepsilon^{5}}
\right)
+h^5\left(
- \frac{\varepsilon^{6} \left(u - u_{1}\right)}{48} - \frac{\varepsilon^{5} \left(4 u - 5 u_{1}\right)}{48} - \frac{\varepsilon^{4} \left(7 u - 12 u_{1}\right)}{48} - \frac{\varepsilon^{3} \left(u - 4 u_{1}\right)}{12} + \frac{\varepsilon^{2} \left(u + 15 u_{1}\right)}{48} + \fra

In [69]:
f1[4].subs(sol).expand().simplify()

19*(-\varepsilon**10*exp(t/\varepsilon) - 5*\varepsilon**9*exp(t/\varepsilon) - 10*\varepsilon**8*exp(t/\varepsilon) - 10*\varepsilon**7*exp(t/\varepsilon) - 5*\varepsilon**6*exp(t/\varepsilon) - \varepsilon**5*exp(t/\varepsilon) - \varepsilon**5*exp(t*(\varepsilon + 2)) + 5*\varepsilon**4*exp(t*(\varepsilon + 2)) - 10*\varepsilon**3*exp(t*(\varepsilon + 2)) + 10*\varepsilon**2*exp(t*(\varepsilon + 2)) - 5*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(720*\varepsilon**5*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [70]:
f2[4].subs(sol).expand().simplify()

19*(\varepsilon**12*exp(t/\varepsilon) + 6*\varepsilon**11*exp(t/\varepsilon) + 15*\varepsilon**10*exp(t/\varepsilon) + 20*\varepsilon**9*exp(t/\varepsilon) + 15*\varepsilon**8*exp(t/\varepsilon) + 6*\varepsilon**7*exp(t/\varepsilon) + \varepsilon**6*exp(t/\varepsilon) - \varepsilon**6*exp(t*(\varepsilon + 2)) + 6*\varepsilon**5*exp(t*(\varepsilon + 2)) - 15*\varepsilon**4*exp(t*(\varepsilon + 2)) + 20*\varepsilon**3*exp(t*(\varepsilon + 2)) - 15*\varepsilon**2*exp(t*(\varepsilon + 2)) + 6*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(720*\varepsilon**6*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [71]:
print(latex(f2[4].subs(sol).expand().simplify()))

\frac{19 \left(\varepsilon^{12} e^{\frac{t}{\varepsilon}} + 6 \varepsilon^{11} e^{\frac{t}{\varepsilon}} + 15 \varepsilon^{10} e^{\frac{t}{\varepsilon}} + 20 \varepsilon^{9} e^{\frac{t}{\varepsilon}} + 15 \varepsilon^{8} e^{\frac{t}{\varepsilon}} + 6 \varepsilon^{7} e^{\frac{t}{\varepsilon}} + \varepsilon^{6} e^{\frac{t}{\varepsilon}} - \varepsilon^{6} e^{t \left(\varepsilon + 2\right)} + 6 \varepsilon^{5} e^{t \left(\varepsilon + 2\right)} - 15 \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} + 20 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} - 15 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} + 6 \varepsilon e^{t \left(\varepsilon + 2\right)} - e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{720 \varepsilon^{6} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


### Adams–Bashforth method 5 order

In [72]:
set_clip(8, 7, Rational(0, 1))
m = AdamsBashforth5(g, Matrix([u(t), u1(t)]))

In [73]:
F1 = clip((T(u(t+5*h))-T(u(t+4*h)))/h - T(m[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-9*(u1_{t} - u_{tt})/2

h^2 =>


...

h^5 =>


-(22583*u1_{ttttt} - 23058*u_{tttttt})/1440

h^6 =>


-(172970*u1_{tttttt} - 185223*u_{ttttttt})/15120

- u_{1} + u_{t}
+h\left(
- \frac{9 \left(u1_{t} - u_{tt}\right)}{2}
\right)
+h^2\left(
- \frac{61 \left(u1_{tt} - u_{ttt}\right)}{6}
\right)
+h^3\left(
- \frac{123 \left(u1_{ttt} - u_{tttt}\right)}{8}
\right)
+h^4\left(
- \frac{2101 \left(u1_{tttt} - u_{ttttt}\right)}{120}
\right)
+h^5\left(
- \frac{22583 u1_{ttttt} - 23058 u_{tttttt}}{1440}
\right)
+h^6\left(
- \frac{172970 u1_{tttttt} - 185223 u_{ttttttt}}{15120}
\right)


In [74]:
F2 = clip((T(u1(t+5*h))-T(u1(t+4*h)))/h - T(m[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


9*\varepsilon*(u1_{t} - u_{t})/2 + 9*u1_{tt}/2 + 9*(u1_{t} + u_{t})/(2*\varepsilon)

h^2 =>


...

h^5 =>


22583*\varepsilon*(u1_{ttttt} - u_{ttttt})/1440 + 1281*u1_{tttttt}/80 + 22583*(u1_{ttttt} + u_{ttttt})/(1440*\varepsilon)

h^6 =>


2471*\varepsilon*(u1_{tttttt} - u_{tttttt})/216 + 61741*u1_{ttttttt}/5040 + 2471*(u1_{tttttt} + u_{tttttt})/(216*\varepsilon)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{9 \varepsilon \left(u1_{t} - u_{t}\right)}{2} + \frac{9 u1_{tt}}{2} + \frac{9 \left(u1_{t} + u_{t}\right)}{2 \varepsilon}
\right)
+h^2\left(
\frac{61 \varepsilon \left(u1_{tt} - u_{tt}\right)}{6} + \frac{61 u1_{ttt}}{6} + \frac{61 \left(u1_{tt} + u_{tt}\right)}{6 \varepsilon}
\right)
+h^3\left(
\frac{123 \varepsilon \left(u1_{ttt} - u_{ttt}\right)}{8} + \frac{123 u1_{tttt}}{8} + \frac{123 \left(u1_{ttt} + u_{ttt}\right)}{8 \varepsilon}
\right)
+h^4\left(
\frac{2101 \varepsilon \left(u1_{tttt} - u_{tttt}\right)}{120} + \frac{2101 u1_{ttttt}}{120} + \frac{2101 \left(u1_{tttt} + u_{tttt}\right)}{120 \varepsilon}
\right)
+h^5\left(
\frac{22583 \varepsilon \left(u1_{ttttt} - u_{ttttt}\right)}{1440} + \frac{1281 u1_{tttttt}}{80} + \frac{22583 \left(u1_{ttttt} + u_{ttttt}\right)}{1440 \varepsilon}
\right)
+h^6\left(
\frac{2471 \varepsilon \left(u1_{tttttt} - u_{tttttt}\right)}{216} + \frac{61741 u1_{t

In [75]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


95*\varepsilon**5*(u - u1)/288 + 95*\varepsilon**4*(3*u - 4*u1)/288 + 95*\varepsilon**3*(u - 2*u1)/72 - 95*\varepsilon**2*u1/36 - 95*\varepsilon*(u + 7*u1)/288 - 95*u/48 + 95*(u - 7*u1)/(288*\varepsilon) + 95*u1/(36*\varepsilon**2) - 95*(u + 2*u1)/(72*\varepsilon**3) + 95*(3*u + 4*u1)/(288*\varepsilon**4) - 95*(u + u1)/(288*\varepsilon**5)

h^6 =>


40763*\varepsilon**6*(u - u1)/60480 + 40763*\varepsilon**5*(4*u - 5*u1)/60480 + 40763*\varepsilon**4*(7*u - 12*u1)/60480 + 40763*\varepsilon**3*(u - 4*u1)/15120 - 40763*\varepsilon**2*(u + 15*u1)/60480 - 40763*\varepsilon*(8*u + 7*u1)/60480 - 40763*u1/7560 - 40763*(8*u - 7*u1)/(60480*\varepsilon) + 40763*(u - 15*u1)/(60480*\varepsilon**2) + 40763*(u + 4*u1)/(15120*\varepsilon**3) - 40763*(7*u + 12*u1)/(60480*\varepsilon**4) + 40763*(4*u + 5*u1)/(60480*\varepsilon**5) - 40763*(u + u1)/(60480*\varepsilon**6)

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{95 \varepsilon^{5} \left(u - u_{1}\right)}{288} + \frac{95 \varepsilon^{4} \left(3 u - 4 u_{1}\right)}{288} + \frac{95 \varepsilon^{3} \left(u - 2 u_{1}\right)}{72} - \frac{95 \varepsilon^{2} u_{1}}{36} - \frac{95 \varepsilon \left(u + 7 u_{1}\right)}{288} - \frac{95 u}{48} + \frac{95 \left(u - 7 u_{1}\right)}{288 \varepsilon} + \frac{95 u_{1}}{36 \varepsilon^{2}} - \frac{95 \left(u + 2 u_{1}\right)}{72 \varepsilon^{3}} + \frac{95 \left(3 u + 4 u_{1}\right)}{288 \varepsilon^{4}} - \frac{95 \left(u + u_{1}\right)}{288 \varepsilon^{5}}
\right)
+h^6\left(
\frac{40763 \varepsilon^{6} \left(u - u_{1}\right)}{60480} + \frac{40763 \varepsilon^{5} \left(4 u - 5 u_{1}\right)}{60480} + \frac{40763 \varepsilon^{4} \left(7 u - 12 u_{1}\right)}{60480} + \frac{40763 \varepsilon^{3} \left(u - 4 u_{1}\right)}{15120} - \frac{40763 \varepsilon^{2} \left(u + 15 u_{1}\right)}{60480} - \frac{4

In [76]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^5 =>


-95*\varepsilon**6*(u - u1)/288 - 95*\varepsilon**5*(4*u - 5*u1)/288 - 95*\varepsilon**4*(7*u - 12*u1)/288 - 95*\varepsilon**3*(u - 4*u1)/72 + 95*\varepsilon**2*(u + 15*u1)/288 + 95*\varepsilon*(8*u + 7*u1)/288 + 95*u1/36 + 95*(8*u - 7*u1)/(288*\varepsilon) - 95*(u - 15*u1)/(288*\varepsilon**2) - 95*(u + 4*u1)/(72*\varepsilon**3) + 95*(7*u + 12*u1)/(288*\varepsilon**4) - 95*(4*u + 5*u1)/(288*\varepsilon**5) + 95*(u + u1)/(288*\varepsilon**6)

h^6 =>


-40763*\varepsilon**7*(u - u1)/60480 - 40763*\varepsilon**6*(5*u - 6*u1)/60480 - 40763*\varepsilon**5*(11*u - 17*u1)/60480 - 40763*\varepsilon**4*(11*u - 28*u1)/60480 - 40763*\varepsilon**3*(3*u - 31*u1)/60480 + 40763*\varepsilon**2*(9*u + 22*u1)/60480 + 40763*\varepsilon*(7*u + 15*u1)/60480 + 40763*u/4320 - 40763*(7*u - 15*u1)/(60480*\varepsilon) + 40763*(9*u - 22*u1)/(60480*\varepsilon**2) + 40763*(3*u + 31*u1)/(60480*\varepsilon**3) - 40763*(11*u + 28*u1)/(60480*\varepsilon**4) + 40763*(11*u + 17*u1)/(60480*\varepsilon**5) - 40763*(5*u + 6*u1)/(60480*\varepsilon**6) + 40763*(u + u1)/(60480*\varepsilon**7)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{95 \varepsilon^{6} \left(u - u_{1}\right)}{288} - \frac{95 \varepsilon^{5} \left(4 u - 5 u_{1}\right)}{288} - \frac{95 \varepsilon^{4} \left(7 u - 12 u_{1}\right)}{288} - \frac{95 \varepsilon^{3} \left(u - 4 u_{1}\right)}{72} + \frac{95 \varepsilon^{2} \left(u + 15 u_{1}\right)}{288} + \frac{95 \varepsilon \left(8 u + 7 u_{1}\right)}{288} + \frac{95 u_{1}}{36} + \frac{95 \left(8 u - 7 u_{1}\right)}{288 \varepsilon} - \frac{95 \left(u - 15 u_{1}\right)}{288 \varepsilon^{2}} - \frac{95 \left(u + 4 u_{1}\right)}{72 \varepsilon^{3}} + \frac{95 \left(7 u + 12 u_{1}\right)}{288 \varepsilon^{4}} - \frac{95 \left(4 u + 5 u_{1}\right)}{288 \varepsilon^{5}} + \frac{95 \left(u + u_{1}\right)}{288 \varepsilon^{6}}
\right)
+h^6\left(
- \frac{40763 \varepsilon^{7} \left(u - u_{1}\right)}{60480} - \frac{40763 \varepsilon^{6}

In [77]:
f1[5].subs(sol).expand().simplify()

95*(-\varepsilon**12*exp(t/\varepsilon) - 6*\varepsilon**11*exp(t/\varepsilon) - 15*\varepsilon**10*exp(t/\varepsilon) - 20*\varepsilon**9*exp(t/\varepsilon) - 15*\varepsilon**8*exp(t/\varepsilon) - 6*\varepsilon**7*exp(t/\varepsilon) - \varepsilon**6*exp(t/\varepsilon) + \varepsilon**6*exp(t*(\varepsilon + 2)) - 6*\varepsilon**5*exp(t*(\varepsilon + 2)) + 15*\varepsilon**4*exp(t*(\varepsilon + 2)) - 20*\varepsilon**3*exp(t*(\varepsilon + 2)) + 15*\varepsilon**2*exp(t*(\varepsilon + 2)) - 6*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(288*\varepsilon**6*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [78]:
f2[5].subs(sol).expand().simplify()

95*(\varepsilon**14*exp(t/\varepsilon) + 7*\varepsilon**13*exp(t/\varepsilon) + 21*\varepsilon**12*exp(t/\varepsilon) + 35*\varepsilon**11*exp(t/\varepsilon) + 35*\varepsilon**10*exp(t/\varepsilon) + 21*\varepsilon**9*exp(t/\varepsilon) + 7*\varepsilon**8*exp(t/\varepsilon) + \varepsilon**7*exp(t/\varepsilon) + \varepsilon**7*exp(t*(\varepsilon + 2)) - 7*\varepsilon**6*exp(t*(\varepsilon + 2)) + 21*\varepsilon**5*exp(t*(\varepsilon + 2)) - 35*\varepsilon**4*exp(t*(\varepsilon + 2)) + 35*\varepsilon**3*exp(t*(\varepsilon + 2)) - 21*\varepsilon**2*exp(t*(\varepsilon + 2)) + 7*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(288*\varepsilon**7*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [79]:
print(latex(f2[5].subs(sol).expand().simplify()))

\frac{95 \left(\varepsilon^{14} e^{\frac{t}{\varepsilon}} + 7 \varepsilon^{13} e^{\frac{t}{\varepsilon}} + 21 \varepsilon^{12} e^{\frac{t}{\varepsilon}} + 35 \varepsilon^{11} e^{\frac{t}{\varepsilon}} + 35 \varepsilon^{10} e^{\frac{t}{\varepsilon}} + 21 \varepsilon^{9} e^{\frac{t}{\varepsilon}} + 7 \varepsilon^{8} e^{\frac{t}{\varepsilon}} + \varepsilon^{7} e^{\frac{t}{\varepsilon}} + \varepsilon^{7} e^{t \left(\varepsilon + 2\right)} - 7 \varepsilon^{6} e^{t \left(\varepsilon + 2\right)} + 21 \varepsilon^{5} e^{t \left(\varepsilon + 2\right)} - 35 \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} + 35 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} - 21 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} + 7 \varepsilon e^{t \left(\varepsilon + 2\right)} - e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{288 \varepsilon^{7} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


### Adams–Moulton method 5 order

In [80]:
set_clip(8, 7, Rational(0, 1))
m = AdamsMoulton5(g, Matrix([u(t), u1(t)]))

In [81]:
F1 = clip((T(u(t+4*h))-T(u(t+3*h)))/h - T(m[0]))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-7*(u1_{t} - u_{tt})/2

h^2 =>


...

h^5 =>


-(6761*u1_{ttttt} - 6734*u_{tttttt})/1440

h^6 =>


-(43232*u1_{tttttt} - 42591*u_{ttttttt})/15120

- u_{1} + u_{t}
+h\left(
- \frac{7 \left(u1_{t} - u_{tt}\right)}{2}
\right)
+h^2\left(
- \frac{37 \left(u1_{tt} - u_{ttt}\right)}{6}
\right)
+h^3\left(
- \frac{175 \left(u1_{ttt} - u_{tttt}\right)}{24}
\right)
+h^4\left(
- \frac{781 \left(u1_{tttt} - u_{ttttt}\right)}{120}
\right)
+h^5\left(
- \frac{6761 u1_{ttttt} - 6734 u_{tttttt}}{1440}
\right)
+h^6\left(
- \frac{43232 u1_{tttttt} - 42591 u_{ttttttt}}{15120}
\right)


In [82]:
F2 = clip((T(u1(t+4*h))-T(u1(t+3*h)))/h - T(m[1]))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


7*\varepsilon*(u1_{t} - u_{t})/2 + 7*u1_{tt}/2 + 7*(u1_{t} + u_{t})/(2*\varepsilon)

h^2 =>


...

h^5 =>


6761*\varepsilon*(u1_{ttttt} - u_{ttttt})/1440 + 3367*u1_{tttttt}/720 + 6761*(u1_{ttttt} + u_{ttttt})/(1440*\varepsilon)

h^6 =>


386*\varepsilon*(u1_{tttttt} - u_{tttttt})/135 + 14197*u1_{ttttttt}/5040 + 386*(u1_{tttttt} + u_{tttttt})/(135*\varepsilon)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
\frac{7 \varepsilon \left(u1_{t} - u_{t}\right)}{2} + \frac{7 u1_{tt}}{2} + \frac{7 \left(u1_{t} + u_{t}\right)}{2 \varepsilon}
\right)
+h^2\left(
\frac{37 \varepsilon \left(u1_{tt} - u_{tt}\right)}{6} + \frac{37 u1_{ttt}}{6} + \frac{37 \left(u1_{tt} + u_{tt}\right)}{6 \varepsilon}
\right)
+h^3\left(
\frac{175 \varepsilon \left(u1_{ttt} - u_{ttt}\right)}{24} + \frac{175 u1_{tttt}}{24} + \frac{175 \left(u1_{ttt} + u_{ttt}\right)}{24 \varepsilon}
\right)
+h^4\left(
\frac{781 \varepsilon \left(u1_{tttt} - u_{tttt}\right)}{120} + \frac{781 u1_{ttttt}}{120} + \frac{781 \left(u1_{tttt} + u_{tttt}\right)}{120 \varepsilon}
\right)
+h^5\left(
\frac{6761 \varepsilon \left(u1_{ttttt} - u_{ttttt}\right)}{1440} + \frac{3367 u1_{tttttt}}{720} + \frac{6761 \left(u1_{ttttt} + u_{ttttt}\right)}{1440 \varepsilon}
\right)
+h^6\left(
\frac{386 \varepsilon \left(u1_{tttttt} - u_{tttttt}\right)}{135} + \frac{14197 u1_{ttt

In [83]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-3*\varepsilon**5*(u - u1)/160 - 3*\varepsilon**4*(3*u - 4*u1)/160 - 3*\varepsilon**3*(u - 2*u1)/40 + 3*\varepsilon**2*u1/20 + 3*\varepsilon*(u + 7*u1)/160 + 9*u/80 - 3*(u - 7*u1)/(160*\varepsilon) - 3*u1/(20*\varepsilon**2) + 3*(u + 2*u1)/(40*\varepsilon**3) - 3*(3*u + 4*u1)/(160*\varepsilon**4) + 3*(u + u1)/(160*\varepsilon**5)

h^6 =>


-281*\varepsilon**6*(u - u1)/12096 - 281*\varepsilon**5*(4*u - 5*u1)/12096 - 281*\varepsilon**4*(7*u - 12*u1)/12096 - 281*\varepsilon**3*(u - 4*u1)/3024 + 281*\varepsilon**2*(u + 15*u1)/12096 + 281*\varepsilon*(8*u + 7*u1)/12096 + 281*u1/1512 + 281*(8*u - 7*u1)/(12096*\varepsilon) - 281*(u - 15*u1)/(12096*\varepsilon**2) - 281*(u + 4*u1)/(3024*\varepsilon**3) + 281*(7*u + 12*u1)/(12096*\varepsilon**4) - 281*(4*u + 5*u1)/(12096*\varepsilon**5) + 281*(u + u1)/(12096*\varepsilon**6)

- u_{1} + u_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{3 \varepsilon^{5} \left(u - u_{1}\right)}{160} - \frac{3 \varepsilon^{4} \left(3 u - 4 u_{1}\right)}{160} - \frac{3 \varepsilon^{3} \left(u - 2 u_{1}\right)}{40} + \frac{3 \varepsilon^{2} u_{1}}{20} + \frac{3 \varepsilon \left(u + 7 u_{1}\right)}{160} + \frac{9 u}{80} - \frac{3 \left(u - 7 u_{1}\right)}{160 \varepsilon} - \frac{3 u_{1}}{20 \varepsilon^{2}} + \frac{3 \left(u + 2 u_{1}\right)}{40 \varepsilon^{3}} - \frac{3 \left(3 u + 4 u_{1}\right)}{160 \varepsilon^{4}} + \frac{3 \left(u + u_{1}\right)}{160 \varepsilon^{5}}
\right)
+h^6\left(
- \frac{281 \varepsilon^{6} \left(u - u_{1}\right)}{12096} - \frac{281 \varepsilon^{5} \left(4 u - 5 u_{1}\right)}{12096} - \frac{281 \varepsilon^{4} \left(7 u - 12 u_{1}\right)}{12096} - \frac{281 \varepsilon^{3} \left(u - 4 u_{1}\right)}{3024} + \frac{281 \varepsilon^{2} \left(u + 15 u_{1}\right)}{12096} + \frac{281 \varepsilon \le

In [84]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^5 =>


3*\varepsilon**6*(u - u1)/160 + 3*\varepsilon**5*(4*u - 5*u1)/160 + 3*\varepsilon**4*(7*u - 12*u1)/160 + 3*\varepsilon**3*(u - 4*u1)/40 - 3*\varepsilon**2*(u + 15*u1)/160 - 3*\varepsilon*(8*u + 7*u1)/160 - 3*u1/20 - 3*(8*u - 7*u1)/(160*\varepsilon) + 3*(u - 15*u1)/(160*\varepsilon**2) + 3*(u + 4*u1)/(40*\varepsilon**3) - 3*(7*u + 12*u1)/(160*\varepsilon**4) + 3*(4*u + 5*u1)/(160*\varepsilon**5) - 3*(u + u1)/(160*\varepsilon**6)

h^6 =>


281*\varepsilon**7*(u - u1)/12096 + 281*\varepsilon**6*(5*u - 6*u1)/12096 + 281*\varepsilon**5*(11*u - 17*u1)/12096 + 281*\varepsilon**4*(11*u - 28*u1)/12096 + 281*\varepsilon**3*(3*u - 31*u1)/12096 - 281*\varepsilon**2*(9*u + 22*u1)/12096 - 281*\varepsilon*(7*u + 15*u1)/12096 - 281*u/864 + 281*(7*u - 15*u1)/(12096*\varepsilon) - 281*(9*u - 22*u1)/(12096*\varepsilon**2) - 281*(3*u + 31*u1)/(12096*\varepsilon**3) + 281*(11*u + 28*u1)/(12096*\varepsilon**4) - 281*(11*u + 17*u1)/(12096*\varepsilon**5) + 281*(5*u + 6*u1)/(12096*\varepsilon**6) - 281*(u + u1)/(12096*\varepsilon**7)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{3 \varepsilon^{6} \left(u - u_{1}\right)}{160} + \frac{3 \varepsilon^{5} \left(4 u - 5 u_{1}\right)}{160} + \frac{3 \varepsilon^{4} \left(7 u - 12 u_{1}\right)}{160} + \frac{3 \varepsilon^{3} \left(u - 4 u_{1}\right)}{40} - \frac{3 \varepsilon^{2} \left(u + 15 u_{1}\right)}{160} - \frac{3 \varepsilon \left(8 u + 7 u_{1}\right)}{160} - \frac{3 u_{1}}{20} - \frac{3 \left(8 u - 7 u_{1}\right)}{160 \varepsilon} + \frac{3 \left(u - 15 u_{1}\right)}{160 \varepsilon^{2}} + \frac{3 \left(u + 4 u_{1}\right)}{40 \varepsilon^{3}} - \frac{3 \left(7 u + 12 u_{1}\right)}{160 \varepsilon^{4}} + \frac{3 \left(4 u + 5 u_{1}\right)}{160 \varepsilon^{5}} - \frac{3 \left(u + u_{1}\right)}{160 \varepsilon^{6}}
\right)
+h^6\left(
\frac{281 \varepsilon^{7} \left(u - u_{1}\right)}{12096} + \frac{281 \varepsilon^{6} \left(5 u - 6 u_{1}\

In [85]:
f1[5].subs(sol).expand().simplify()

3*(\varepsilon**12*exp(t/\varepsilon) + 6*\varepsilon**11*exp(t/\varepsilon) + 15*\varepsilon**10*exp(t/\varepsilon) + 20*\varepsilon**9*exp(t/\varepsilon) + 15*\varepsilon**8*exp(t/\varepsilon) + 6*\varepsilon**7*exp(t/\varepsilon) + \varepsilon**6*exp(t/\varepsilon) - \varepsilon**6*exp(t*(\varepsilon + 2)) + 6*\varepsilon**5*exp(t*(\varepsilon + 2)) - 15*\varepsilon**4*exp(t*(\varepsilon + 2)) + 20*\varepsilon**3*exp(t*(\varepsilon + 2)) - 15*\varepsilon**2*exp(t*(\varepsilon + 2)) + 6*\varepsilon*exp(t*(\varepsilon + 2)) - exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(160*\varepsilon**6*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [86]:
f2[5].subs(sol).expand().simplify()

3*(-\varepsilon**14*exp(t/\varepsilon) - 7*\varepsilon**13*exp(t/\varepsilon) - 21*\varepsilon**12*exp(t/\varepsilon) - 35*\varepsilon**11*exp(t/\varepsilon) - 35*\varepsilon**10*exp(t/\varepsilon) - 21*\varepsilon**9*exp(t/\varepsilon) - 7*\varepsilon**8*exp(t/\varepsilon) - \varepsilon**7*exp(t/\varepsilon) - \varepsilon**7*exp(t*(\varepsilon + 2)) + 7*\varepsilon**6*exp(t*(\varepsilon + 2)) - 21*\varepsilon**5*exp(t*(\varepsilon + 2)) + 35*\varepsilon**4*exp(t*(\varepsilon + 2)) - 35*\varepsilon**3*exp(t*(\varepsilon + 2)) + 21*\varepsilon**2*exp(t*(\varepsilon + 2)) - 7*\varepsilon*exp(t*(\varepsilon + 2)) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(160*\varepsilon**7*(-exp(1/\varepsilon) + exp(\varepsilon + 2)))

In [87]:
print(latex(f2[5].subs(sol).expand().simplify()))

\frac{3 \left(- \varepsilon^{14} e^{\frac{t}{\varepsilon}} - 7 \varepsilon^{13} e^{\frac{t}{\varepsilon}} - 21 \varepsilon^{12} e^{\frac{t}{\varepsilon}} - 35 \varepsilon^{11} e^{\frac{t}{\varepsilon}} - 35 \varepsilon^{10} e^{\frac{t}{\varepsilon}} - 21 \varepsilon^{9} e^{\frac{t}{\varepsilon}} - 7 \varepsilon^{8} e^{\frac{t}{\varepsilon}} - \varepsilon^{7} e^{\frac{t}{\varepsilon}} - \varepsilon^{7} e^{t \left(\varepsilon + 2\right)} + 7 \varepsilon^{6} e^{t \left(\varepsilon + 2\right)} - 21 \varepsilon^{5} e^{t \left(\varepsilon + 2\right)} + 35 \varepsilon^{4} e^{t \left(\varepsilon + 2\right)} - 35 \varepsilon^{3} e^{t \left(\varepsilon + 2\right)} + 21 \varepsilon^{2} e^{t \left(\varepsilon + 2\right)} - 7 \varepsilon e^{t \left(\varepsilon + 2\right)} + e^{t \left(\varepsilon + 2\right)}\right) e^{- \varepsilon t + \varepsilon - t + 1 - \frac{t}{\varepsilon} + \frac{1}{\varepsilon}}}{160 \varepsilon^{7} \left(- e^{\frac{1}{\varepsilon}} + e^{\varepsilon + 2}\right)}


In [88]:
end = time.time()
print(end - start)

112.93402791023254
